##**Part 1: Build an n-gram language model**

In [163]:
#loading necessary libraries
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.data.path.append('.')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Load Data**

In [164]:
# Read the Input file
with open("eng_news_2020_10K-sentences-1.txt", "r") as file:
  input_data = file.readlines()

In [165]:
# Split the line at the first space to remove the index and space
data = [line.split(maxsplit=1)[1] if line.split(maxsplit=1)[0].isdigit() else line for line in input_data]

In [166]:
# Print few lines from the file and convert to String format
print(f"Sentence: {data[0:30]}")
data = "".join(data)

Sentence: ['“18 months ago, we expelled a boy at Nations for selling drugs in six schools.\n', '” 41 Nigeria Centre for Disease Control (NCDC) staff and 17 World Health Organisation (WHO) staff are deployed at the moment to support the Kano response.\n', '⏰8.00pm ⚽️Liverpool v Arsenal Watch the match live at The Arch on six screens with surround sound commentary!\n', 'A 13-year veteran of the department, he worked his way up the ranks of the department starting as an ambulance paramedic at Station 49, then going to the SFFD Academy and graduating as a paramedic and firefighter.\n', 'A 1975 class ring from Robert E. Lee High School in Houston was found near the bones.\n', 'A 2014 resurgence in Polio when there were 359 reported cases of wild poliomyelitis, spread over 12 countries including Pakistan.\n', '“A 30% correction sounds scary right, but the Dow, S&P 500 and the Nasdaq rallied to nearly 30% in just one year last year.\n', 'A 33-year-old Bothwell man was arrested and charged wit

**Data preprocessing**

In [167]:
# Func to split the Input data to sentences
def split_to_sentences(data):
    sentences = data.split("\n")
    sentences = [s.strip() for s in sentences]
    sentences = [s for s in sentences if len(s) > 0]
    return sentences

In [168]:
# Func to tokenize the sentences - Setter
def tokenize_sentences(sentences):
    tokenized_sentences = []
    for sentence in sentences:
        sentence = sentence.lower()
        tokenized = nltk.word_tokenize(sentence)
        tokenized_sentences.append(tokenized)
    return tokenized_sentences

In [169]:
# Func to tokenize the sentences - Getter
def get_tokenized_data(data):
    sentences = split_to_sentences(data)
    tokenized_sentences = tokenize_sentences(sentences)
    return tokenized_sentences

In [170]:
# Func to split data into training and test
tokenized_data = get_tokenized_data(data)
random.seed(87)
random.shuffle(tokenized_data)

train_size = int(len(tokenized_data) * 0.8)
train_data = tokenized_data[0:train_size]
test_data = tokenized_data[train_size:]

In [171]:
# Func to get count of words
def count_words(tokenized_sentences):
    word_counts = {}

    for sentence in tokenized_sentences:
        for token in sentence:
            if token not in word_counts:
                word_counts[token] = 1
            else:
                word_counts[token] += 1

    return word_counts

In [172]:
# Func to handle unknown words
def replace_more_freq_words_by_unk(tokenized_sentences, vocabulary, unknown_token="<unk>"):
    vocabulary = set(vocabulary)
    replaced_tokenized_sentences = []
    for sentence in tokenized_sentences:
        replaced_sentence = []
        for token in sentence:
            if token in vocabulary:
                replaced_sentence.append(token)
            else:
                replaced_sentence.append(unknown_token)
        replaced_tokenized_sentences.append(replaced_sentence)

    return replaced_tokenized_sentences

In [173]:
# Func to find the words that appears more than the set threshold
def get_words_with_nplus_frequency(tokenized_sentences, count_threshold):
    closed_vocab = []
    word_counts = count_words(tokenized_sentences)
    for word, cnt in word_counts.items():
        if cnt >=count_threshold:
            closed_vocab.append(word)
    return closed_vocab

In [174]:
# Func to pre-process the data - Handles unknown
def preprocess_data(
    train_data,
    test_data,
    count_threshold,
    unknown_token="<unk>",
    get_words_with_nplus_frequency=get_words_with_nplus_frequency,
    replace_more_freq_words_by_unk=replace_more_freq_words_by_unk
    ):

    vocabulary = get_words_with_nplus_frequency(train_data,count_threshold)

    train_data_replaced = replace_more_freq_words_by_unk(train_data,vocabulary)
    test_data_replaced = replace_more_freq_words_by_unk(test_data,vocabulary)

    return train_data_replaced, test_data_replaced, vocabulary

**Preprocess the data**

In [175]:
# It handles the Unknown words
train_data, test_data, _= preprocess_data(
    train_data,
    test_data,
    count_threshold = 1
    )

In [176]:
# Func to create n-gram of the data
def count_n_grams(
    data,
    n,
    start_token='<s>',
    end_token = '</s>'
    ):

    n_grams = {}
    for sentence in data:
        sentence = [start_token]*n + sentence +[end_token]
        for i in range(len(sentence) - n + 1):
            n_gram = tuple(sentence[i:i+n])

            if n_gram in n_grams:
                n_grams[n_gram] += 1
            else:
                n_grams[n_gram] = 1

    return n_grams

In [177]:
# Bigram
# Printing 10 bigram from test data
n = 2
count = 10
for item in count_n_grams(test_data, n):
  if count ==0:
    break
  print(item)
  count-=1

('<s>', '<s>')
('<s>', 'speaking')
('speaking', 'with')
('with', '<unk>')
('<unk>', ',')
(',', 'the')
('the', 'former')
('former', 'heavyweight')
('heavyweight', '<unk>')
('<unk>', 'said')


In [178]:
# Trigram
# Printing 10 bigram from test data
n = 3
count = 10
for item in count_n_grams(test_data, n):
  if count ==0:
    break
  count -=1
  print(item)

('<s>', '<s>', '<s>')
('<s>', '<s>', 'speaking')
('<s>', 'speaking', 'with')
('speaking', 'with', '<unk>')
('with', '<unk>', ',')
('<unk>', ',', 'the')
(',', 'the', 'former')
('the', 'former', 'heavyweight')
('former', 'heavyweight', '<unk>')
('heavyweight', '<unk>', 'said')


In [179]:
# Func to estimate probablity with k Laplace Smoothing
def estimate_probability(
    word, previous_n_gram,
    n_gram_counts,
    n_plus1_gram_counts,
    vocabulary_size,
    k=1.0
    ):

    previous_n_gram = tuple(previous_n_gram)
    previous_n_gram_count = n_gram_counts.get(previous_n_gram, 0)
    denominator = previous_n_gram_count + k * vocabulary_size

    n_plus1_gram = previous_n_gram + (word,)

    n_plus1_gram_count = n_plus1_gram_counts.get(n_plus1_gram, 0)

    numerator = n_plus1_gram_count + k
    probability = numerator/denominator
    return probability

In [180]:
# Func to get the list of all estimated probabilities
def estimate_probabilities(
    previous_n_gram,
    n_gram_counts,
    n_plus1_gram_counts,
    vocabulary,
    end_token='</s>',
    unknown_token="<unk>",
    k=1.0
    ):

    previous_n_gram = tuple(previous_n_gram)
    #print('Handling the unknown token')
    vocabulary = vocabulary + [end_token, unknown_token]
    vocabulary_size = len(vocabulary)
    probabilities = {}
    for word in vocabulary:
        probability = estimate_probability(word, previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary_size, k=k)

        probabilities[word] = probability
    return probabilities

In [181]:
# Func to generate count matrix
def make_count_matrix(
    n_plus1_gram_counts,
    vocabulary
    ):

    vocabulary = vocabulary + ["</s>", "<unk>"]

    n_grams = []
    for n_plus1_gram in n_plus1_gram_counts.keys():
        n_gram = n_plus1_gram[0:-1]
        n_grams.append(n_gram)
    n_grams = list(set(n_grams))

    row_index = {n_gram:i for i, n_gram in enumerate(n_grams)}
    col_index = {word:j for j, word in enumerate(vocabulary)}

    nrow = len(n_grams)
    ncol = len(vocabulary)
    count_matrix = np.zeros((nrow, ncol))
    for n_plus1_gram, count in n_plus1_gram_counts.items():
        n_gram = n_plus1_gram[0:-1]
        word = n_plus1_gram[-1]
        if word not in vocabulary:
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i, j] = count

    count_matrix = pd.DataFrame(count_matrix, index=n_grams, columns=vocabulary)
    return count_matrix

**Probability Matrix**

In [182]:
# Func to generate probability matrix
def make_probability_matrix(
    n_plus1_gram_counts,
    vocabulary,
    k
    ):
    count_matrix = make_count_matrix(n_plus1_gram_counts, unique_words)
    count_matrix += k
    prob_matrix = count_matrix.div(count_matrix.sum(axis=1), axis=0)
    return prob_matrix

In [183]:
sentences = list(test_data[0]+test_data[1])
bigram_count = count_n_grams(test_data[0:2], 2)
unique_words = list(set(sentences))
print("Bi-Gram Probability Matrix")
display(make_probability_matrix(bigram_count, unique_words, k=2))

Bi-Gram Probability Matrix


,black,charleston,too,did,is,",",uniform,speaking,n't,.,...,democrat,for,simple,fact,largely,said,was,considered,</s>,<unk>
"(:,)",0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472
"(at,)",0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472
"(did,)",0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.033708,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472
"(speaking,)",0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472
"(represents,)",0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472
"(<unk>,)",0.021739,0.021739,0.021739,0.021739,0.021739,0.032609,0.021739,0.021739,0.021739,0.021739,...,0.021739,0.021739,0.021739,0.021739,0.021739,0.032609,0.021739,0.021739,0.021739,0.021739
"(former,)",0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472
"(near,)",0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472
"(said,)",0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472
"(“,)",0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472


In [184]:
sentences = list(test_data[0]+test_data[1])
trigram_counts = count_n_grams(test_data[0:1], 3)
unique_words = list(set(sentences))
print("Tri-Gram Probability Matrix")
display(make_probability_matrix(trigram_counts, unique_words, k=2))

Tri-Gram Probability Matrix


,black,charleston,too,did,is,",",uniform,speaking,n't,.,...,democrat,for,simple,fact,largely,said,was,considered,</s>,<unk>
"(<unk>, said)",0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472
"(did, n't)",0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472
"(uniform, was)",0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472
"(<s>, <s>)",0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.033708,0.022472,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472
"(fact, is)",0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472
"(speaking, with)",0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.033708
"(simple, fact)",0.022472,0.022472,0.022472,0.022472,0.033708,0.022472,0.022472,0.022472,0.022472,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472
"(way, too)",0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472
"(heavy, for)",0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472
"(all, ,)",0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,...,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472,0.022472


##**Part 2: Implement Sentence Generation**

In [185]:
# Func to suggest the next word
def suggest_a_word(
    previous_tokens,
    n_gram_counts,
    n_plus1_gram_counts,
    vocabulary,
    end_token='</s>',
    unknown_token="<unk>",
    k=1.0,
    start_with=None
    ):


    n = len(list(n_gram_counts.keys())[0])

    previous_tokens = ['<s>'] * n + previous_tokens

    previous_n_gram = previous_tokens[-n:]

    probabilities = estimate_probabilities(previous_n_gram,
                                           n_gram_counts, n_plus1_gram_counts,
                                           vocabulary, k=k)


    suggestion = None

    max_prob = 0

    for word, prob in probabilities.items():
        if start_with is not None:
            if not word.startswith(start_with):
                continue

        if prob > max_prob:
            suggestion = word
            max_prob = prob

    return suggestion, max_prob

In [186]:
# Func for Sentence generation
def generate_sentence(
    n_gram_counts,
    n_plus1_gram_counts,
    vocabulary,
    start_token='<s>',
    end_token = '</s>',
    k=1.0
    ):


    # length of previous words
    n = len(list(n_gram_counts.keys())[0])

    # initialize previous n-gram with start tokens
    previous_n_gram = [start_token]*n
    sentence = previous_n_gram.copy()

    # generate words until end token is found
    while True:
        # get the most probable next word
        next_word, prob = suggest_a_word(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=k)

        if next_word == end_token:
            break

        sentence.append(next_word)

        # update previous n-gram
        previous_n_gram = sentence[len(sentence)-n:len(sentence)]

    return ' '.join(sentence[n:])

In [187]:
# Using test data
vocab = []
for i in range(0,10):
  vocab += test_data[i]

print(f"Before generating sentences: {vocab}")

Before generating sentences: ['speaking', 'with', '<unk>', ',', 'the', 'former', 'heavyweight', '<unk>', 'said', ':', '“', 'he', 'did', "n't", 'hurt', 'me', 'at', 'all', ',', 'but', 'the', 'simple', 'fact', 'is', 'my', 'uniform', 'was', 'way', 'too', 'heavy', 'for', 'me', '.', '<unk>', 'represents', 'the', 'largely', 'black', '<unk>', 'in', 'and', 'near', 'columbia', 'and', 'charleston', ',', 'and', 'he', 'is', 'considered', 'the', 'most', 'powerful', 'democrat', 'in', 'the', 'state', '.', 'the', 'markets', '’', '<unk>', 'signal', 'traders', 'think', 'the', 'odds', 'of', 'a', 'contested', 'election', 'are', '<unk>', ',', 'and', 'that', 'there', '’', 's', 'a', 'path', 'toward', 'renewed', 'us', 'government', 'stimulus', 'to', 'support', 'the', 'economy', '.', 'the', 'people', 'in', 'your', 'household', 'will', 'also', 'need', 'to', 'restrict', 'their', 'movements', '.', 'the', 'black', 'lives', 'matter', 'protests', 'that', 'filled', 'the', 'streets', 'of', 'australia', '’', 's', 'citie

In [188]:
# Generate a sentence
bigram_count = count_n_grams(test_data[0:2], 2)
trigram_count = count_n_grams(test_data[0:2],3)
sentence = generate_sentence(bigram_count, trigram_count, vocab)
print(sentence)

speaking with <unk> , the former heavyweight <unk> said : “ he did n't hurt me at all , but the simple fact is my uniform was way too heavy for me .


In [189]:
bigram_count = count_n_grams(test_data[2:5], 2)
trigram_count = count_n_grams(test_data[2:5],3)
sentence = generate_sentence(bigram_count, trigram_count, vocab)
print(sentence)

the black lives matter protests that filled the streets of australia ’ s a path toward renewed us government stimulus to support the economy .


In [190]:
# Custom example to test the model for Sentence Generation
bigram_count = {('I', 'am'): 2, ('am', 'a'): 1, ('a', 'student'): 1, ('<s>', 'I'): 1, ('student', '</s>'): 1}
trigram_count = {('I', 'am', 'a'): 1, ('am', 'a', 'student'): 1, ('<s>', 'I', 'am'): 1, ('a', 'student', '</s>'): 1}
vocabulary = ['I', 'am', 'a', 'student', '<s>', '</s>']

sentence = generate_sentence(bigram_count, trigram_count, vocabulary)
print(sentence)

I am a student
